## Source code for _Investigating particle size-flux relationships and the biological pump across a range of plankton ecosystem states from coastal to oligotrophic_

Manuscript by: __Christian K. Fender, Thomas B. Kelly, Lionel Guidi, Mark D. Ohman, Matthew C. Smith, and Michael R. Stukel__

Code by: __Thomas B. Kelly and Christian K. Fender (FSU)__

Manuscript submitted to Frontiers in Marine Science (March 2019)


### _Abstract_

Sinking particles transport organic carbon produced in the surface ocean to the ocean interior, leading to net storage of atmospheric CO2 in the deep ocean.  The rapid growth of in situ imaging technology has the potential to revolutionize our understanding of particle flux attenuation in the ocean; however, estimating particle flux from particle size and abundance (measured directly by in situ cameras) is challenging. Sinking rates are dependent on several factors, including particle excess density and porosity, which vary based on particle origin and type. Additionally, particle characteristics are transformed while sinking. We compare optically-measured particle size spectra profiles (Underwater Vision Profiler 5, UVP) with contemporaneous measurements of particle flux made using sediment traps and $^{234}Th:^{238}U$ disequilibrium on six process cruises from the California Current Ecosystem (CCE) LTER Program. These measurements allow us to assess the efficacy of using size-flux relationships to estimate fluxes from optical particle size measurements. We find that previously published parameterizations that estimate carbon flux from UVP profiles are a poor fit to direct flux measurements in the CCE.  This discrepancy is found to result primarily from the important role of fecal pellets in particle flux.  These pellets are primarily in a size range (i.e., 100 – 400 µm) that is not well-resolved as images by the UVP due to the resolution of the sensor.  We develop a new, CCE-optimized algorithm for estimating carbon flux from UVP data in the southern California Current (Flux = $\sum_{i=1}^{x}{n_i A d_i^B ∆d_i }$), with A = 13.45, B = 1.35, d = particle diameter (mm) and Flux in units of $mg C m^{-2} d^{-1}$.  We caution, however, that increased accuracy in flux estimates derived from optical instruments will require devices with greater resolution, the ability to differentiate fecal pellets from low porosity marine snow aggregates, and improved sampling of rapidly sinking fecal pellets.  We also find that the particle size-flux relationships may be different within the euphotic zone than in the shallow twilight zone and hypothesize that the changing nature of sinking particles with depth must be considered when investigating the remineralization length scale of sinking particles in the ocean.


---

### Note on downloading the Raw Data

An account with Ecotaxa will be necessary for the following steps.

1. From the home page, select the drop down window next to Action at the top right. Select the Particle Module.
2. Enter the relevant filters. For this study all CCELTER Particle Projects were selected.
3. Select Export Selection.
4. Download the RAW data format. Only this format will work with the provided code.
5. From the downloaded files, only the PAR files and the metadata summary file were used for analysis.

In [ ]:
## 